In [1]:
import os
import json

In [2]:

def preprocess_years(years):
    
    if "Board" in years:
        return [0]
    if "CBSE" in years:
        return [0]
    if "IGCSE" in years:
        return [0]
    if "ICSE" in years:
        return [0]
    years=years.replace("Class","")
    years=years.replace("Pre-Nursery","Nursery")
    years=years.replace("-",":")
    years=years.replace("Nursery","-2")
    years=years.replace("Pre School","-2")
    years=years.replace("LKG","-1")
    years=years.replace("UKG","0")
    years=years.replace("KG","-1")
    years=years.replace("PRE","0")
    
    
    start_grade,end_grade=years.split(":")

    
    

    years_list = []
    if start_grade is not None and end_grade is not None:
        years_list.extend(range(int(start_grade.strip()), int(end_grade.strip())+1))
    return years_list


In [3]:
def preprocess_fee(fee):
    if fee is None or fee.strip() == '':
        return [-1]  # Return 0 if fee is missing or empty
    if fee=="Free":
        return [0]
    og=fee
    # Remove unwanted characters
    fee = fee.replace(',', '')
    fee = fee.replace("Fee range is from Rs. ","")
    fee = fee.replace("The annual fee to be paid stands at ", "")
    fee = fee.replace("The total fee for Nur is Rs. ", "")
    fee = fee.replace("for Class 10 is Rs. ", "-")
    fee = fee.replace("Rs. ", "")
    fee = fee.replace(" lakhs", "00000")
    fee = fee.replace(" lakh", "00000")
    fee = fee.replace('â‚¹', '')
    fee = fee.replace(' Lakhs', '00000')
    fee = fee.replace("to", "-")
    fee = fee.replace("/ Annual", "")
    fee = fee.replace("Annually", "")
    fee=fee.replace("/-","")
    fee=fee.replace("/","")
    fee = fee.replace(".", "")
    fee=fee.replace(" ","")
    fee=fee.replace("Annual","")
    fee=fee.replace("year","")

    # Split by '-' if it's a range
    fee_parts = fee.strip().split('-')
    if len(fee_parts) == 1:
        # If only one part, it's a single value, so convert it to integer
        try:
            return [int(fee_parts[0].strip())]
        except Exception as e:
            print(f"Invalid fee format1: {fee}", fee_parts)
            return [-1]
    elif len(fee_parts) == 2:
        # If two parts, it's a range, so convert both parts to integers and return as a list
        try:
            return [int(fee_parts[0].strip()), int(fee_parts[1].strip())]
        except Exception as e:
            if fee_parts[-1]=="":
                return [int(fee_parts[0].strip())]
            else:
                print(f"Invalid fee format2: {fee}", fee_parts)
            return [-1]
    elif len(fee_parts) == 4:
        try:
            return [int(fee_parts[0][:-1].strip()), int(fee_parts[2][:-1].strip())]
        except Exception as e:
            print(f"Invalid fee format3: {fee}", fee_parts)
            return [-1]
    else:
        print(f"Invalid fee format: {og}")
        return [-1]  # Default value if fee is not in the correct format


In [4]:
def preprocess_since(since):
    if since==None:
        return 0
    if since.isdigit():
        return int(since)
    
    return 0

In [14]:
def logic(file_name):
    with open(file_name, 'r') as file:
            data_json= json.load(file)
    preprocessed_data = []
    board_set=set()
    since_li=set()
    string_li=["Category","Faculty","Sports","Amenities","Board"]
    count=0
    for school in data_json:
        
        try:
            if school["Name"].isdigit() or school['Years']=="":
                count+=1
                continue
            preprocessed_school = {
                "Name": school["Name"],
                "Category": school["Category"],
                "Location": school["Location"],
                "Faculty": school["Faculty"],
                "Sports": school["Sports"],
                "Amenities": school["Amenities"],
                "Board": school["Board"],
                
            }
            is_none=lambda s: "" if s==None else s
            is_cat=lambda s: "Other" if s.isdigit() or s=="N/A" or s=="" else s
            preprocessed_school['Category']=is_cat(preprocessed_school["Category"])
            
            preprocessed_school["Years"] = preprocess_years(school["Years"])
            preprocessed_school["Fee"] = preprocess_fee(school["Fee"])
            is_number = lambda s: int(s) if s.isdigit() else 0
            preprocessed_school['Since']=is_number(school['Since'])
            preprocessed_school['Strength']=is_number(school['Strength'])
            board_set.add(preprocessed_school['Category'])
            for i in preprocessed_school:
                if preprocessed_school[i]==None or preprocessed_school[i]=="N/A":
                    if i in string_li:
                        preprocessed_school[i]=""
                    else:
                        print(preprocessed_school)
            if len(preprocessed_school["Years"])==1:
                preprocessed_school["Board"]=school["Years"]
                #print(preprocessed_school["Board"])
            preprocessed_data.append(preprocessed_school)
        except Exception as e:
            print(f"Preprocessing failed for entry: {school}")
            print(f"Error message: {str(e)}")
            break
    print(board_set)

    
    if len(preprocessed_data) == len(data_json):
        area=file_name.replace("all_schools_data","")
        area=area.replace("_","")
        area=area.replace(".json","")
        output_file =f'Preprocessed_Output\\preprocessed_data_{area}.json'
        with open(output_file, 'w') as file:
            json.dump(preprocessed_data, file, indent=4)
        print(f"Preprocessed data has been written to {output_file}")
    else:
        print("Not same")
        print(len(preprocessed_data),len(data_json))
        area=file_name.replace("all_schools_data","")
        area=area.replace("_","")
        area=area.replace(".json","")
        output_file =f'Preprocessed_Output\\preprocessed_data_{area}.json'
        with open(output_file, 'w') as file:
            json.dump(preprocessed_data, file, indent=4)
        print(f"Preprocessed data has been written to {output_file}")
    print(count)

In [7]:


# Get the current directory
current_directory = os.getcwd()

# List all files in the current directory
files_in_directory = os.listdir(current_directory)

# Filter JSON files
json_files = [file for file in files_in_directory if file.endswith('.json')]
print(json_files)

['all_schools_data.json', 'all_schools_data_banjara-hills.json', 'all_schools_data_begumpet.json', 'all_schools_data_dilsukhnagar.json', 'all_schools_data_himayatnagar.json', 'all_schools_data_jubilee-hills.json', 'all_schools_data_kondapur.json', 'all_schools_data_kukatpally.json', 'all_schools_data_madhapur.json', 'all_schools_data_miyapur.json', 'all_schools_data_patancheruvu.json', 'all_schools_data_secunderabad.json', 'all_schools_data_uppal.json']


In [15]:

for file_name in json_files:
    logic(file_name)

{'Play Schools', 'Play Schools, Other', 'International Schools, Other Collections', 'Public Schools, Boarding Schools', 'Day Schools', 'Govt Schools', 'International Schools, Public Schools', 'Other Collections', 'Public Schools', 'International Schools, Boarding Schools', 'Special Education Schools', 'International Schools', 'Other', 'Play Schools, Public Schools', 'International Schools, Play Schools', 'Special Education Schools, Public Schools'}
Preprocessed data has been written to Preprocessed_Output\preprocessed_data_.json
0
{'Play Schools', 'Special Education Schools, Public Schools', 'International Schools, Other Collections', 'Play Schools, Other', 'Public Schools, Boarding Schools', 'Day Schools', 'Govt Schools', 'International Schools, Public Schools', 'Other Collections', 'Public Schools', 'Special Education Schools', 'International Schools', 'Other', 'Play Schools, Public Schools', 'International Schools, Play Schools', 'Day cum Boarding Schools', 'Public Schools, Other Co